# Toepassing PPO

In [1]:
#!pip install gym
#!pip install stable_baselines3
#!pip install gymnasium[atari]

In [ ]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO

In [2]:
class PPOAgent:
    """
    Proximal Policy Optimization (PPO) agent for the Atari Warlords environment.

    Attributes:
        env_name (str): Name of the gym environment.
        env: The instantiated gym environment.
        model: The PPO model from stable-baselines3.
    """

    def __init__(self, env_name="ALE/Warlords-v5"):
        """
        Initializes the PPOAgent with a specified environment.

        Parameters:
            env_name (str): Gym environment name. Defaults to "ALE/Warlords-v5".
        """
        self.env = gym.make(env_name)
        # CnnPolicy, want de input zijn afbeeldingen
        self.model = PPO(
            "CnnPolicy", 
            self.env, 
            verbose=1,
            learning_rate=2.5e-4,
            gamma=0.99,
            n_steps=128,
            batch_size=256,
            ent_coef=0.01
        )
        self.env_name = env_name

    def train(self, timesteps=100_000):
        """
        Train the PPO model for a specified number of timesteps.

        Parameters:
            timesteps (int): Number of training timesteps.
        """
        self.model.learn(total_timesteps=timesteps)

    def act(self, observation):
        """
        Selects an action based on the current observation using the trained model.

        Parameters:
            observation (np.ndarray): Current environment observation.

        Returns:
            int: Chosen action.
        """
        action, _ = self.model.predict(observation, deterministic=True)
        return action

    def save(self, path="ppo_warlords.zip"):
        """
        Saves the trained model to a file.

        Parameters:
            path (str): Path to save the model.
        """
        self.model.save(path)

    def load(self, path="ppo_warlords.zip"):
        """
        Loads a trained model from a file.

        Parameters:
            path (str): Path to the saved model.
        """
        self.model = PPO.load(path, env=self.env)

    def evaluate(self, episodes=5):
        """
        Evaluates the PPO agent for a given number of episodes and prints the total reward.

        Parameters:
            episodes (int): Number of evaluation episodes.
        """
        env = gym.make(self.env_name)
        for ep in range(episodes):
            obs = env.reset()
            done = False
            total_reward = 0
            while not done:
                action = self.act(obs)
                obs, reward, done, info = env.step(action)
                total_reward += reward
                env.render()
            print(f"Episode {ep+1}: Reward = {total_reward}")
        env.close()

C:\Users\bosch\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\bosch\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Uitleg van PPO

Proximal Policy Optimization (PPO) is een geavanceerd reinforcement learning-algoritme dat veel wordt gebruikt voor het trainen van agents in complexe omgevingen zoals Atari-spellen. PPO verbetert eerdere policy-gradient-methodes door bij elke stap de aanpassing van het beleid (“policy”) te beperken. Hierdoor wordt het leerproces stabieler en is de kans kleiner dat de agent ineens “vergeet” wat hij geleerd heeft.

PPO werkt door het beleid steeds een klein beetje aan te passen op basis van ervaringen uit de omgeving. Hierdoor leert de agent efficiënter en zijn de resultaten vaak beter reproduceerbaar. PPO is ook geschikt voor situaties met hoge-dimensionale input, zoals beelden, en werkt goed in multi-agent omgevingen zoals Warlords.

DhanushKumar (2024)

### Motivatie

Voor deze opdracht heb ik gekozen voor het algoritme Proximal Policy Optimization (PPO). De belangrijkste reden hiervoor is dat PPO bekend staat om zijn stabiliteit en efficiëntie bij het trainen van agents in complexe omgevingen met hoge-dimensionale input, zoals de Atari-game Warlords, Schulman et al. (2017). Omdat PPO de aanpassingen aan het beleid per stap beperkt, blijft het leerproces gecontroleerd en voorkom je dat de agent tijdens het trainen “vergeet” wat eerder geleerd is. Dit is vooral belangrijk in multi-agent omgevingen, waar het gedrag van andere agents de situatie voortdurend beïnvloedt.

Daarnaast is PPO eenvoudig te implementeren dankzij bestaande libraries zoals stable-baselines3, waardoor het mogelijk is om snel te experimenteren met verschillende hyperparameters. De standaardwaarden die ik voor de belangrijkste hyperparameters heb gekozen, zijn gebaseerd op aanbevelingen uit de literatuur en eerdere succesvolle toepassingen in vergelijkbare omgevingen (The 37 Implementation Details Of Proximal Policy Optimization · The ICLR Blog Track, 2022). PPO is bovendien goed schaalbaar, waardoor het geschikt is om in een multi-agent setting zoals Warlords verschillende agents onafhankelijk van elkaar te trainen en te vergelijken.

Door deze eigenschappen is PPO naar mijn mening de meest geschikte keuze voor deze opdracht, omdat het zorgt voor betrouwbare leerresultaten en flexibiliteit biedt bij het uitvoeren van experimenten met verschillende agents en trainingsinstellingen.

### Aanpak

Voor deze opdracht heb ik een PPO-agent geïmplementeerd met behulp van de stable-baselines3 library. PPO is gekozen vanwege de stabiele prestaties en de robuustheid bij het trainen in omgevingen met beeldinput, zoals “Warlords”. De agent gebruikt een convolutioneel neuraal netwerk om de observaties te verwerken.
De belangrijkste hyperparameters zijn gekozen op basis van aanbevolen waarden uit wetenschappelijke literatuur voor Atari-omgevingen, maar kunnen verder geoptimaliseerd worden.
Mijn implementatie maakt het makkelijk om het model op te slaan, opnieuw te laden en te evalueren, zodat experimenten goed reproduceerbaar zijn.

### Hyperparameters

##### Overzicht keuzes

| Hyperparameter        | Waarde    | Uitleg                                                                                 |
|----------------------|-----------|----------------------------------------------------------------------------------------|
| `Policy`              | CnnPolicy | Gebruikt convolutieneurale netwerken, geschikt voor beeldinput zoals bij Atari-games   |
| `learning_rate`        | 2.5e-4    | Standaardwaarde voor PPO in Atari-omgevingen; zorgt voor een goede balans tussen snel leren en stabiliteit |
| `gamma`                | 0.99      | Kortingfactor; waardeert toekomstige beloningen bijna net zo hoog als directe beloning |
| `n_steps`              | 128       | Aantal stappen per update; bepaalt hoeveel ervaringen per batch worden verzameld       |
| `batch_size`           | 256       | Grootte van elke batch die wordt gebruikt tijdens het updaten van het beleid           |
| `ent_coef (entropie)`  | 0.01      | Coëfficiënt voor exploratie; hogere waarde zorgt voor meer exploratiegedrag            |
| `verbose`              | 1         | Zorgt voor gedetailleerde logging tijdens het trainen                                  |


##### Experimenteren met hyperparameters

Tijdens het trainen heb ik geëxperimenteerd met verschillende waarden voor de learning rate, entropie-coëfficiënt en batch size:
- Learning rate: Een hogere learning rate zorgde voor snellere training, maar maakte het model soms instabiel. 2.5e-4 bleek een goed compromis.
- Entropie-coëfficiënt (ent_coef): Met een hogere waarde ging de agent meer experimenteren, maar duurde het langer voordat hij goed leerde. 0.01 gaf een goede balans.
- Batch size: Grotere batches zorgden voor stabielere updates, maar vroegen meer geheugen.

De uiteindelijke hyperparameterkeuzes zijn gebaseerd op wat het beste werkte voor deze specifieke omgeving en op basis van literatuur.

### Literatuur

- DhanushKumar. (2024, 12 mei). PPO Algorithm - DhanushKumar - Medium. Medium. https://medium.com/@danushidk507/ppo-algorithm-3b33195de14a
- Schulman, J., Wolski, F., Dhariwal, P., Radford, A., & Klimov, O. (2017, 20 juli). Proximal Policy optimization Algorithms. arXiv.org. https://arxiv.org/abs/1707.06347
- The 37 Implementation Details of Proximal Policy Optimization · The ICLR Blog Track. (2022, 25 maart). https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/?utm_source=chatgpt.com